In [ ]:
# default_exp core

# Library Documentation

> API details for DashComponentBase, DashFigureFactory, DashComponent and DashApp

In [ ]:
#hide
from nbdev.showdoc import *

## imports

In [ ]:
#export

import sys
from abc import ABC
import inspect
import types
from importlib import import_module

import shortuuid
import oyaml as yaml

import dash
import jupyter_dash

import dash_bootstrap_components as dbc

In [ ]:
#export
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

## DashComponentBase

All classes in the library are derived from `DashComponentBase` which is a an abstract base class (ABC) that provides:

- automatic storing of parameters to attributes
- automatic storing of parameters to a `._stored_params` property
- exporting component to a config dict with `.to_config()`
- exporting component to a yaml file with `.to_yaml()`
- building component from a config dict with classmethod `.from_config()`
- building component from a yaml file with classmethod `.from_yaml()`


In [ ]:
#export
class DashComponentBase(ABC):  
    """Base class for all dash_oop_components classes. 
    
    Stores parameter of child classes to attributes and ._stored_params.
    Proved .to_config(), to_yaml(), .from_config() and .from_yaml() methods
    """
    def __init__(self, no_store=None, no_attr=None, no_config=None, child_depth=3):
        """
        Args:
            no_store {list, bool}: either a list of parameters not to store or True, in which
                case no parameters gets stored.
            no_attr {list, bool}: either a list of parameter not to assign to attribute or True,
                in which case no parameters get assigned to attributes
            no_config {list, bool}: either a list of parameter not to store to ._stored_params
                or True, in which case no parameters get saved to ._stored_params
            child_depth (int): how deep the child is from which parameters will be read.
                Defaults to 3 (i.e. The child of the child of DashComponentBase)
        """
        self._store_child_params(no_store, no_attr, no_config, child_depth)

    def _store_child_params(self, no_store=None, no_attr=None, no_config=None, child_depth=3):
        """
        Args:
            no_store {list, bool}: either a list of parameters not to store or True, in which
                case no parameters gets stored.
            no_attr {list, bool}: either a list of parameter not to assign to attribute or True,
                in which case no parameters get assigned to attributes
            no_config {list, bool}: either a list of parameter not to store to ._stored_params
                or True, in which case no parameters get saved to ._stored_params
            child_depth (int): how deep the child is from which parameters will be read.
                Defaults to 3 (i.e. The child of the child of DashComponentBase)
        """
             
        if not hasattr(self, '_stored_params'): 
            self._stored_params = {}
            
        child_frame = sys._getframe(child_depth)
        child_args = child_frame.f_code.co_varnames[1:child_frame.f_code.co_argcount]
        child_dict = {arg: child_frame.f_locals[arg] for arg in child_args}
        if 'kwargs' in child_frame.f_locals:
            child_dict['kwargs'] = child_frame.f_locals['kwargs']
        
        if isinstance(no_store, bool) and no_store:
            return
        else:
            if no_store is None: no_store = tuple()
        
        if isinstance(no_attr, bool) and no_attr: dont_attr = True
        else:
            if no_attr is None: no_attr = tuple()
            dont_attr = False 
            
        if isinstance(no_config, bool) and no_param: dont_config = True
        else:
            if no_config is None: no_config= tuple()
            dont_config = False 

        for name, value in child_dict.items():
            if name in {'dash_component', 'dash_figure_factory', 'dash_app'}:
                raise ValueError(f"Please do not use {name} as a parameter name, "
                                 "as this results in a confusing and hard to parse config.")
            if not dont_attr and name not in no_store and name not in no_attr:
                setattr(self, name, value)
            if not dont_config and name not in no_store and name not in no_config:
                self._stored_params[name] = value
      
    def to_config(self):
        """
        returns a dict with class name, module and params 
        """
        return dict(dash_component=dict(
            name=self.__class__.__name__, 
            module=self.__class__.__module__,
            params=self._stored_params))
        
    def to_yaml(self, filepath=None):
        """
        stores a yaml configuration to disk. 
        
        If no filepath is given, returns a str of the yaml config.
        """
        yaml_config = self.to_config()
        if filepath is not None:
            yaml.dump(yaml_config, open(filepath, "w"))
            return
        return yaml.dump(yaml_config)
    
    @classmethod
    def from_config(cls, config, **update_params):
        """
        Loads a dash_oop_component class from a configuration dict.
        
        Args:
            config (dict): configuration dict, generated from .to_config()
            update_params: a dict of parameters to be overridden by update_params
        
        Returns:
            Instance of the class defined in the config.
        """
        if 'dash_component' in config:
            config = config['dash_component']
        elif 'dash_figure_factory' in config:
            config = config['dash_figure_factory']
        elif 'dash_app' in config:
            config = config['dash_app']
        else:
            raise ValueError("I only know how to build dash_component, "
                             "dash_figure_factory and dash_app from config!", str(config))
        
        params = config['params']
        if not params: params = {}
            
        for k, v in update_params.items():
            if k in params:
                params[k] = v
            elif 'kwargs' in params:
                params['kwargs'][k]=v
            else:
                raise ValueError(f"This dash_oop_component does not take {k} as an argument, "
                                "nor does it take **kwargs!")
        
        for k, v in params.items():
            if isinstance(v, dict) and ('dash_figure_factory' in v or 'dash_component' in v):
                params[k] = DashComponentBase.from_config(v)
        
        component_class = getattr(import_module(config['module']), config['name'])
        if 'kwargs' in params:
            kwargs = params.pop('kwargs')
        else:
            kwargs = {}
        return component_class(**params, **kwargs)
    
    @classmethod
    def from_yaml(cls, yaml_filepath, **update_params):
        """
        Loads a dash_oop_component class from a yaml file.
        
        Args:
            yaml_filepath (str, Path): filepath of a .yaml file, generated from .to_yaml()
            update_params: a dict of parameters to be overridden by update_params
        
        Returns:
            Instance of the class defined in the yaml file.
        """
        config = yaml.safe_load(open(str(yaml_filepath), "r"))
        return cls.from_config(config, **update_params)
        

In [ ]:
show_doc(DashComponentBase.to_config)
show_doc(DashComponentBase.to_yaml)
show_doc(DashComponentBase.from_config)
show_doc(DashComponentBase.from_config)

<h4 id="DashComponentBase.to_config" class="doc_header"><code>DashComponentBase.to_config</code><a href="__main__.py#L68" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponentBase.to_config</code>()

returns a dict with class name, module and params 

<h4 id="DashComponentBase.to_yaml" class="doc_header"><code>DashComponentBase.to_yaml</code><a href="__main__.py#L77" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponentBase.to_yaml</code>(**`filepath`**=*`None`*)

stores a yaml configuration to disk. 

If no filepath is given, returns a str of the yaml config.

<h4 id="DashComponentBase.from_config" class="doc_header"><code>DashComponentBase.from_config</code><a href="__main__.py#L89" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponentBase.from_config</code>(**`config`**, **\*\*`update_params`**)

Loads a dash_oop_component class from a configuration dict.

Args:
    config (dict): configuration dict, generated from .to_config()
    update_params: a dict of parameters to be overridden by update_params

Returns:
    Instance of the class defined in the config.

<h4 id="DashComponentBase.from_config" class="doc_header"><code>DashComponentBase.from_config</code><a href="__main__.py#L89" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponentBase.from_config</code>(**`config`**, **\*\*`update_params`**)

Loads a dash_oop_component class from a configuration dict.

Args:
    config (dict): configuration dict, generated from .to_config()
    update_params: a dict of parameters to be overridden by update_params

Returns:
    Instance of the class defined in the config.

### Example use of DashComponentBase

In [ ]:
class T(DashComponentBase): 
    def __init__(self, a=1, b=2, **kwargs):
        super().__init__(child_depth=2)
        
t = T(a=2, b=3)

configuration dict:

In [ ]:
print(t.to_config())

{'dash_component': {'name': 'T', 'module': '__main__', 'params': {'a': 2, 'b': 3, 'kwargs': {}}}}


yaml configuration:

In [ ]:
print(t.to_yaml())

dash_component:
  name: T
  module: __main__
  params:
    a: 2
    b: 3
    kwargs: {}



Tests showing everything works as expected

In [ ]:
assert t.a == 2
assert t.b == 3
assert t.to_config()['dash_component']['name'] == "T"
assert t.to_config()['dash_component']['module'] == "__main__"
assert t.to_config()['dash_component']['params']['a'] == t.a
assert t.to_config()['dash_component']['params']['b'] == t.b

Store to yaml:

In [ ]:
t.to_yaml("T.yaml")

Load from yaml and check that loaded instance `t2` is same as `t`:

In [ ]:
t2 = T.from_yaml("T.yaml")

assert t2.to_config()['dash_component']['name'] == "T"
assert t2.to_config()['dash_component']['module'] == "__main__"
assert t2.to_config()['dash_component']['params']['a'] == t.a
assert t2.to_config()['dash_component']['params']['b'] == t.b

Override parameters upon loading:

In [ ]:
t2 = T.from_yaml("T.yaml", b=4)
assert t2.b == 4

If `**kwargs` in the class definition than unknown parameters get loads to `self.kwargs`:

In [ ]:
t2 = T.from_yaml("T.yaml", c=5)
assert hasattr(t2, "kwargs")
assert "c" in t2.kwargs
assert t2.kwargs["c"] == 5

## DashFigureFactory

A `DashFigureFactory` loads data and provides plots, tables, lists, dicts that can be used for building your dashboard.

This provides a clean seperation where all your data preparation and visualisation logic goes into one place (the `DashFigureFactory`) and all the dashboard layout and interaction logic goes into another (the `DashComponents`).

When used as a parameter to a `DashComponent` (e.g. parameter `figure_factory`)the parameter `figure_factory` gets replaced by the figure factory configuration dict. This means that a `DashComponent` automatically loads its underlying `DashFigureFactory` parameters upon load.

In [ ]:
#export

class DashFigureFactory(DashComponentBase):
    """
    Helper class to store data for a dashboard and provide e.g. plotting functions.
    
    You should seperate the datastorage/plotting logic from the dashboard logic.
    All data/plotting logic goes into a DashFigureFactory.
    
    All dashboard logic goes into a DashComponent.
    
    Stores to config under key 'dash_figure_factory'
    """
    def __init__(self, no_store=None, no_attr=None, no_config=None):
        super().__init__(no_store=None, no_attr=None, no_config=None)
        
    def to_config(self):
        return dict(dash_figure_factory=dict(
            name=self.__class__.__name__, 
            module=self.__class__.__module__,
            params=self._stored_params))
        
    

In [ ]:
show_doc(DashFigureFactory)

<h2 id="DashFigureFactory" class="doc_header"><code>class</code> <code>DashFigureFactory</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>DashFigureFactory</code>(**`no_store`**=*`None`*, **`no_attr`**=*`None`*, **`no_config`**=*`None`*) :: [`DashComponentBase`](/dash_oop_components/core.html#DashComponentBase)

Helper class to store data for a dashboard and provide e.g. plotting functions.

You should seperate the datastorage/plotting logic from the dashboard logic.
All data/plotting logic goes into a DashFigureFactory.

All dashboard logic goes into a DashComponent.

Stores to config under key 'dash_figure_factory'

### Example use of `DashFigureFactory`

A DashFigureFactory that simply stores a list and has a `return_list()` method:

In [ ]:
class ListFactory(DashFigureFactory):
    def __init__(self, list_input):
        super().__init__()
        
    def list_length(self):
        return len(self.list_input)
        
    def return_list(self, first_n_items=None):
        if first_n_items is None: first_n_items = self.list_length()
        if first_n_items <= len(self.list_input):
            return self.list_input[:first_n_items]
        return None

In [ ]:
lf = ListFactory(["this", "is", "a", "dumb", "example"])
print(lf.return_list())

['this', 'is', 'a', 'dumb', 'example']


In [ ]:
print(lf.to_config())

{'dash_figure_factory': {'name': 'ListFactory', 'module': '__main__', 'params': {'list_input': ['this', 'is', 'a', 'dumb', 'example']}}}


In [ ]:
print(lf.to_yaml())

dash_figure_factory:
  name: ListFactory
  module: __main__
  params:
    list_input:
    - this
    - is
    - a
    - dumb
    - example



In [ ]:
assert lf.list_input == ['this', 'is', 'a', 'dumb', 'example']
assert lf.return_list() == ['this', 'is', 'a', 'dumb', 'example']
assert lf.return_list(2) == ['this', 'is']

Storing and loading from yaml:

In [ ]:
lf.to_yaml("ff.yaml")

In [ ]:
lf2 = DashFigureFactory.from_yaml("ff.yaml")

assert lf2.list_input == lf.list_input
assert lf2.return_list() == lf.return_list()
assert lf2.return_list(2) == lf.return_list(2)



## DashComponent

A `DashComponent` combines a `dash` layout with `dash` callbacks. 

It provides:
- a `.layout()` method that returns the layout for the components
- a `.register_callbacks(app)` method that allow you to register the 
    callbacks of the component to a specific dash app
- automatic conversion of any `DashFigureFactory` parameters to it's config 
    (this means that figure factory will be automatically loaded upon load)
- automatic registration of all `DashComponent` subcomponents in its attributes
    (which means that callbacks of all subcomponents will also automatically be registered)
- a `make_hideable` staticmethod that makes it easy to hide parts of a layout 
    depending on configuration bools
    
It inherits from `DashComponentBase` so:
- All parameters are automatically stored to attributes and to ._stored_params
- Can be exported and loaded from config and yaml


In [ ]:
#export

class DashComponent(DashComponentBase):
    """DashComponent is a bundle of a dash layout and callbacks that
    can make use of DashFigureFactory objects. 

    A DashComponent can have DashComponent subcomponents, that
    you register with register_components(). 

    DashComponents allow you to:

    1. Write clean, re-usable, composable code for your dashboard
    2. Store your dashboard to config files
    3. Load your dashboard from config files

    Each DashComponent should have a unique .name so that dash id's don't clash.
    If no name is given, DashComponent generates a unique uuid name. This allows
    for multiple instance of the same component type in a single layout. 
    But remember to add `+self.name` to all id's.

    Important:
        define your callbacks in `_register_callbacks()` (note underscore!) and
        DashComponent will register callbacks of subcomponents in addition
        to _register_callbacks() when calling register_callbacks()
    """
    def __init__(self, title="Dash", name=None, 
                 no_store=None, no_attr=None, no_config=None):
        """initialize the DashComponent

        Args:
            title (str, optional): Title of component. Defaults to "Dash".
            name (str, optional): unique name to add to Component elements. 
                        If None then random uuid is generated to make sure 
                        it's unique. Defaults to None.
        """
        super().__init__(no_store, no_attr, no_config)
        self._convert_ff_config_params()
        
        self.title = title
        self.name = name
        if self.name is None:
            self.name = str(shortuuid.ShortUUID().random(length=10))

        self._components = []
        
    def _convert_ff_config_params(self):
        """convert any DashFigureFactory in the ._stored_params dict to its config"""
        for k, v in self._stored_params.items():
            if isinstance(v, DashFigureFactory):
                self._stored_params[k] = self._stored_params[k].to_config()
                
    @staticmethod
    def make_hideable(element, hide=False):
        """helper function to optionally not display an element in a layout.        

        Example:
            make_hideable(dbc.Col([cutoff.layout()]), hide=hide_cutoff)

        Args:
            hide(bool): wrap the element inside a hidden html.div. If the element 
                        is a dbc.Col or a dbc.FormGroup, wrap element.children in
                        a hidden html.Div instead. Defaults to False.
        """ 
        if hide:
            if isinstance(element, dbc.Col) or isinstance(element, dbc.FormGroup):
                return html.Div(element.children, style=dict(display="none"))
            else:
                return html.Div(element, style=dict(display="none"))
        else:
            return element
        
    def register_components(self): 
        """register subcomponents so that their callbacks will be registered
        
        Searches self.__dict__, finds all DashComponents and adds them to self._components
        """
        if not hasattr(self, '_components'):
            self._components = []
        for comp in self.__dict__.values():
            if isinstance(comp, DashComponent) and comp not in self._components:
                self._components.append(comp)
    
    def layout(self):
        """layout to be defined by the particular ExplainerComponent instance.
        All element id's should append +self.name to make sure they are unique."""
        return None

    def _register_callbacks(self, app):
        """register callbacks specific to this ExplainerComponent."""
        pass

    def register_callbacks(self, app):
        """First register callbacks of all subcomponents, then call
        _register_callbacks(app)
        """
        self.register_components()
        for comp in self._components:
            comp.register_callbacks(app)
        self._register_callbacks(app)

In [ ]:
show_doc(DashComponent)
show_doc(DashComponent.make_hideable)
show_doc(DashComponent.layout)
show_doc(DashComponent._register_callbacks)
show_doc(DashComponent.register_callbacks)

<h2 id="DashComponent" class="doc_header"><code>class</code> <code>DashComponent</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>DashComponent</code>(**`title`**=*`'Dash'`*, **`name`**=*`None`*, **`no_store`**=*`None`*, **`no_attr`**=*`None`*, **`no_config`**=*`None`*) :: [`DashComponentBase`](/dash_oop_components/core.html#DashComponentBase)

DashComponent is a bundle of a dash layout and callbacks that
can make use of DashFigureFactory objects. 

A DashComponent can have DashComponent subcomponents, that
you register with register_components(). 

DashComponents allow you to:

1. Write clean, re-usable, composable code for your dashboard
2. Store your dashboard to config files
3. Load your dashboard from config files

Each DashComponent should have a unique .name so that dash id's don't clash.
If no name is given, DashComponent generates a unique uuid name. This allows
for multiple instance of the same component type in a single layout. 
But remember to add `+self.name` to all id's.

Important:
    define your callbacks in `_register_callbacks()` (note underscore!) and
    DashComponent will register callbacks of subcomponents in addition
    to _register_callbacks() when calling register_callbacks()

<h4 id="DashComponent.make_hideable" class="doc_header"><code>DashComponent.make_hideable</code><a href="__main__.py#L52" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.make_hideable</code>(**`element`**, **`hide`**=*`False`*)

helper function to optionally not display an element in a layout.        

Example:
    make_hideable(dbc.Col([cutoff.layout()]), hide=hide_cutoff)

Args:
    hide(bool): wrap the element inside a hidden html.div. If the element 
                is a dbc.Col or a dbc.FormGroup, wrap element.children in
                a hidden html.Div instead. Defaults to False.

<h4 id="DashComponent.layout" class="doc_header"><code>DashComponent.layout</code><a href="__main__.py#L83" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.layout</code>()

layout to be defined by the particular ExplainerComponent instance.
All element id's should append +self.name to make sure they are unique.

<h4 id="DashComponent._register_callbacks" class="doc_header"><code>DashComponent._register_callbacks</code><a href="__main__.py#L88" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent._register_callbacks</code>(**`app`**)

register callbacks specific to this ExplainerComponent.

<h4 id="DashComponent.register_callbacks" class="doc_header"><code>DashComponent.register_callbacks</code><a href="__main__.py#L92" class="source_link" style="float:right">[source]</a></h4>

> <code>DashComponent.register_callbacks</code>(**`app`**)

First register callbacks of all subcomponents, then call
_register_callbacks(app)

### Example use of `DashComponent`:


In [ ]:
import dash_html_components as html
import dash_core_components as dcc

#### Single DashComponent

Each `DashComponent` gets assigned a random uuid identifier `self.name`. These can be added to all elements in the layout to make sure the layout of each component is unique.

In [ ]:
class ListComponent(DashComponent):
    def __init__(self, list_factory, first_n=2):
        super().__init__()
        
    def layout(self):
        return html.Div([
            dcc.Input(
                id="input-first-n-"+self.name, 
                type="number", 
                value=self.first_n,
                min=0,
                max=self.list_factory.list_length()),
            html.Div(id="output-div-"+self.name, children=" ".join(self.list_factory.return_list(self.first_n))),
        ])
    
    def _register_callbacks(self, app):
        @app.callback(
            Output("output-div-"+self.name, "children"),
            Input("input-first-n-"+self.name, "value")
        )
        def update_div(first_n):
            if first_n is not None:
                return " ".join(self.list_factory.return_list(first_n))
            raise PreventUpdate

In [ ]:
lc = ListComponent(lf)

In [ ]:
assert lc.list_factory.return_list() == ['this', 'is', 'a', 'dumb', 'example']
assert isinstance(lc.layout(), html.Div)

In [ ]:
print(lc.to_config())

{'dash_component': {'name': 'ListComponent', 'module': '__main__', 'params': {'list_factory': {'dash_figure_factory': {'name': 'ListFactory', 'module': '__main__', 'params': {'list_input': ['this', 'is', 'a', 'dumb', 'example']}}}, 'first_n': 2}}}


In [ ]:
print(lc.to_yaml())

dash_component:
  name: ListComponent
  module: __main__
  params:
    list_factory:
      dash_figure_factory:
        name: ListFactory
        module: __main__
        params:
          list_input:
          - this
          - is
          - a
          - dumb
          - example
    first_n: 2



Store and reload (with the figure factory automatically getting reloaded as well!)

In [ ]:
lc.to_yaml("list_component.yaml")

In [ ]:
lc2 = ListComponent.from_yaml("list_component.yaml")

In [ ]:
print(lc2.to_yaml())

dash_component:
  name: ListComponent
  module: __main__
  params:
    list_factory:
      dash_figure_factory:
        name: ListFactory
        module: __main__
        params:
          list_input:
          - this
          - is
          - a
          - dumb
          - example
    first_n: 2



In [ ]:
assert lc2.list_factory.return_list() == lc.list_factory.return_list()

#### Compose multiple DashComponent from sub components

A `ListComposite` if a combination of two `ListComponents`, both with different initial settings for `first_n`.

- the subcomponents get defined in the init with `self.list1 = ...` and `self.list2 = ...`
- the subcomponents get included in the layout by including `self.list1.layout()` and `self.list2.layout()`
- callbacks can be written that include elements from the ListComposite and the subcomponents
    (in this case the reset button resets the two inputs of the subcomponents)
- additional callbacks should be defined under `_register_callbacks(self, app)`! (**note the underscore!**)
- callbacks of the subcomponents also get automatically registered, and included when you call
    `register_components(app)` (**note the lack of underscore!**)

In [ ]:
class ListComposite(DashComponent):
    def __init__(self, list_factory, first_n1=2, first_n2=3):
        super().__init__()
        self.list1 = ListComponent(self.list_factory, first_n=first_n1)
        self.list2 = ListComponent(self.list_factory, first_n=first_n2)
        
    def layout(self):
        return html.Div([
            html.Button("Reset", id="reset-button-"+self.name),
            self.list1.layout(),
            self.list2.layout()
        ])
    
    def _register_callbacks(self, app):
        @app.callback(
            Output("input-first-n-"+self.list1.name, "value"),
            Output("input-first-n-"+self.list2.name, "value"),
            Input("reset-button-"+self.name, "n_clicks")
        )
        def reset_inpus(n_clicks):
            if n_clicks:
                return self.first_n1, self.first_n2
            raise PreventUpdate

In [ ]:
list_composite = ListComposite(lf)

In [ ]:
print(list_composite.to_yaml())

dash_component:
  name: ListComposite
  module: __main__
  params:
    list_factory:
      dash_figure_factory:
        name: ListFactory
        module: __main__
        params:
          list_input:
          - this
          - is
          - a
          - dumb
          - example
    first_n1: 2
    first_n2: 3



subcomponents are automatically registerd to a `_components` list when calling `register_components()`.

(this gets called initially in `register_callbacks()`)

In [ ]:
list_composite.register_components()
list_composite._components
assert len(list_composite._components) == 2

In [ ]:
list_composite.layout()

Div([Button(children='Reset', id='reset-button-9vEKZcaMqr'), Div([Input(id='input-first-n-6dSiwTWZAB', value=2, type='number', max=5, min=0), Div(children='this is', id='output-div-6dSiwTWZAB')]), Div([Input(id='input-first-n-J9rmvcx2XU', value=3, type='number', max=5, min=0), Div(children='this is a', id='output-div-J9rmvcx2XU')])])

When we build a dash app from this list_composite, we can check that all three callbacks (one for the composite, and one each for each subcomponent) have indeed been registered:

In [ ]:
app = dash.Dash()
app.layout = list_composite.layout()
list_composite.register_callbacks(app)
assert len(app.callback_map) == 3

## DashApp

In [ ]:
#export

def concat_docstring(source=None):
    "Decorator: `__doc__` from `source` to __doc__"
    def _f(f):
        if isinstance(f, types.FunctionType):
            from_f = f
        else:
            from_f = f.__init__
            
        if isinstance(source, types.FunctionType):
            source_f = source
        elif source.__init__.__doc__ is not None:
            source_f = source.__init__
        else:
            source_f = source
        from_f.__doc__ = (
            str(from_f.__doc__) + 
            "\n\n\-----------------------\n\n" + 
            f"Docstring from {source.__name__}" +
            "\n\n" +
            str(source_f.__doc__))
        return f
    return _f

In [ ]:
#export

class DashApp(DashComponentBase):
    """Wrapper class for dash apps. Assigns layout and callbacks from 
    a DashComponent to a Dash app, and runs it.
    
    Can run both Dash and JupyterDash apps.
    
    """
    @concat_docstring(dash.Dash)
    def __init__(self, dashboard_component, port=8050, mode='dash', **kwargs):
        """
        
        Args:
            dashboard_component (DashComponent): component to be run
            port (int): port to run the server
            mode ({'dash', 'external', 'inline', 'jupyterlab'}): type of dash server to start
            kwargs: all kwargs will be passed down to dash.Dash. See below the docstring of dash.Dash
            
        Returns:
            DashApp: simply start .run() to start the dashboard
        """
        super().__init__(child_depth=2)
        self._stored_params['dashboard_component'] = dashboard_component.to_config()
        self.app = self._get_dash_app()
                
    def _get_dash_app(self):
        if self.mode == 'dash':
            app = dash.Dash(**self.kwargs)
        elif self.mode in {'inline', 'external', 'jupyterlab'}:
            app = jupyter_dash.JupyterDash(**self.kwargs)
            
        app.layout = self.dashboard_component.layout()
        self.dashboard_component.register_callbacks(app)
        app.title = self.dashboard_component.title
        return app
    
    def to_config(self):
        return dict(dash_app=dict(
            name=self.__class__.__name__, 
            module=self.__class__.__module__,
            params=self._stored_params))
    
    def flask_server(self):
        """returns flask server inside self.app, for building wsgi apps"""
        return self.app.server
    
    def run(self, port=None):
        """Run the dash app"""
        self.app.run_server(port=port if port is not None else self.port)
        

In [ ]:
show_doc(DashApp)
show_doc(DashApp.__init__)
show_doc(DashApp.flask_server)
show_doc(DashApp.run)

<h2 id="DashApp" class="doc_header"><code>class</code> <code>DashApp</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>DashApp</code>(**`dashboard_component`**, **`port`**=*`8050`*, **`mode`**=*`'dash'`*, **\*\*`kwargs`**) :: [`DashComponentBase`](/dash_oop_components/core.html#DashComponentBase)

Wrapper class for dash apps. Assigns layout and callbacks from 
a DashComponent to a Dash app, and runs it.

Can run both Dash and JupyterDash apps.

<h4 id="DashApp.__init__" class="doc_header"><code>DashApp.__init__</code><a href="__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>DashApp.__init__</code>(**`dashboard_component`**, **`port`**=*`8050`*, **`mode`**=*`'dash'`*, **\*\*`kwargs`**)

        
        Args:
            dashboard_component (DashComponent): component to be run
            port (int): port to run the server
            mode ({'dash', 'external', 'inline', 'jupyterlab'}): type of dash server to start
            kwargs: all kwargs will be passed down to dash.Dash. See below the docstring of dash.Dash
            
        Returns:
            DashApp: simply start .run() to start the dashboard
        

\-----------------------

Docstring from Dash

Dash is a framework for building analytical web applications.
    No JavaScript required.

    If a parameter can be set by an environment variable, that is listed as:
        env: ``DASH_****``
    Values provided here take precedence over environment variables.

    :param name: The name Flask should use for your app. Even if you provide
        your own ``server``, ``name`` will be used to help find assets.
        Typically ``__name__`` (the magic global var, not a string) is the
        best value to use. Default ``'__main__'``, env: ``DASH_APP_NAME``
    :type name: string

    :param server: Sets the Flask server for your app. There are three options:
        ``True`` (default): Dash will create a new server
        ``False``: The server will be added later via ``app.init_app(server)``
            where ``server`` is a ``flask.Flask`` instance.
        ``flask.Flask``: use this pre-existing Flask server.
    :type server: boolean or flask.Flask

    :param assets_folder: a path, relative to the current working directory,
        for extra files to be used in the browser. Default ``'assets'``.
        All .js and .css files will be loaded immediately unless excluded by
        ``assets_ignore``, and other files such as images will be served if
        requested.
    :type assets_folder: string

    :param assets_url_path: The local urls for assets will be:
        ``requests_pathname_prefix + assets_url_path + '/' + asset_path``
        where ``asset_path`` is the path to a file inside ``assets_folder``.
        Default ``'assets'``.
    :type asset_url_path: string

    :param assets_ignore: A regex, as a string to pass to ``re.compile``, for
        assets to omit from immediate loading. Ignored files will still be
        served if specifically requested. You cannot use this to prevent access
        to sensitive files.
    :type assets_ignore: string

    :param assets_external_path: an absolute URL from which to load assets.
        Use with ``serve_locally=False``. Dash can still find js and css to
        automatically load if you also keep local copies in your assets
        folder that Dash can index, but external serving can improve
        performance and reduce load on the Dash server.
        env: ``DASH_ASSETS_EXTERNAL_PATH``
    :type assets_external_path: string

    :param include_assets_files: Default ``True``, set to ``False`` to prevent
        immediate loading of any assets. Assets will still be served if
        specifically requested. You cannot use this to prevent access
        to sensitive files. env: ``DASH_INCLUDE_ASSETS_FILES``
    :type include_assets_files: boolean

    :param url_base_pathname: A local URL prefix to use app-wide.
        Default ``'/'``. Both `requests_pathname_prefix` and
        `routes_pathname_prefix` default to `url_base_pathname`.
        env: ``DASH_URL_BASE_PATHNAME``
    :type url_base_pathname: string

    :param requests_pathname_prefix: A local URL prefix for file requests.
        Defaults to `url_base_pathname`, and must end with
        `routes_pathname_prefix`. env: ``DASH_REQUESTS_PATHNAME_PREFIX``
    :type requests_pathname_prefix: string

    :param routes_pathname_prefix: A local URL prefix for JSON requests.
        Defaults to ``url_base_pathname``, and must start and end
        with ``'/'``. env: ``DASH_ROUTES_PATHNAME_PREFIX``
    :type routes_pathname_prefix: string

    :param serve_locally: If ``True`` (default), assets and dependencies
        (Dash and Component js and css) will be served from local URLs.
        If ``False`` we will use CDN links where available.
    :type serve_locally: boolean

    :param compress: Use gzip to compress files and data served by Flask.
        Default ``True``
    :type compress: boolean

    :param meta_tags: html <meta> tags to be added to the index page.
        Each dict should have the attributes and values for one tag, eg:
        ``{'name': 'description', 'content': 'My App'}``
    :type meta_tags: list of dicts

    :param index_string: Override the standard Dash index page.
        Must contain the correct insertion markers to interpolate various
        content into it depending on the app config and components used.
        See https://dash.plotly.com/external-resources for details.
    :type index_string: string

    :param external_scripts: Additional JS files to load with the page.
        Each entry can be a string (the URL) or a dict with ``src`` (the URL)
        and optionally other ``<script>`` tag attributes such as ``integrity``
        and ``crossorigin``.
    :type external_scripts: list of strings or dicts

    :param external_stylesheets: Additional CSS files to load with the page.
        Each entry can be a string (the URL) or a dict with ``href`` (the URL)
        and optionally other ``<link>`` tag attributes such as ``rel``,
        ``integrity`` and ``crossorigin``.
    :type external_stylesheets: list of strings or dicts

    :param suppress_callback_exceptions: Default ``False``: check callbacks to
        ensure referenced IDs exist and props are valid. Set to ``True``
        if your layout is dynamic, to bypass these checks.
        env: ``DASH_SUPPRESS_CALLBACK_EXCEPTIONS``
    :type suppress_callback_exceptions: boolean

    :param prevent_initial_callbacks: Default ``False``: Sets the default value
        of ``prevent_initial_call`` for all callbacks added to the app.
        Normally all callbacks are fired when the associated outputs are first
        added to the page. You can disable this for individual callbacks by
        setting ``prevent_initial_call`` in their definitions, or set it
        ``True`` here in which case you must explicitly set it ``False`` for
        those callbacks you wish to have an initial call. This setting has no
        effect on triggering callbacks when their inputs change later on.

    :param show_undo_redo: Default ``False``, set to ``True`` to enable undo
        and redo buttons for stepping through the history of the app state.
    :type show_undo_redo: boolean

    :param plugins: Extend Dash functionality by passing a list of objects
        with a ``plug`` method, taking a single argument: this app, which will
        be called after the Flask server is attached.
    :type plugins: list of objects

    :param title: Default ``Dash``. Configures the document.title
    (the text that appears in a browser tab).

    :param update_title: Default ``Updating...``. Configures the document.title
    (the text that appears in a browser tab) text when a callback is being run.
    Set to None or '' if you don't want the document.title to change or if you
    want to control the document.title through a separate component or
    clientside callback.
    

<h4 id="DashApp.flask_server" class="doc_header"><code>DashApp.flask_server</code><a href="__main__.py#L44" class="source_link" style="float:right">[source]</a></h4>

> <code>DashApp.flask_server</code>()

returns flask server inside self.app, for building wsgi apps

<h4 id="DashApp.run" class="doc_header"><code>DashApp.run</code><a href="__main__.py#L48" class="source_link" style="float:right">[source]</a></h4>

> <code>DashApp.run</code>(**`port`**=*`None`*)

Run the dash app

### Example use of `DashApp`

You can build and run dash app by simply passing a `DashComposite` to `DashApp` and then running it:

In [ ]:
db = DashApp(list_composite)

- You can set the port with `port=8051`
- You can run the dashboard inline in a notebook by pasing `mode='inline'`
    - or `mode='external'` or `mode='jupyterlab'`
    - default is `mode='dash'`
- Any additional parameters will be passed on the to `dash.Dash()` constructor

In [ ]:
#hide 
run_app = False

In [ ]:
if run_app:
    db.run()

You can also store and reload an entire dashboard:

In [ ]:
print(db.to_yaml())

dash_app:
  name: DashApp
  module: __main__
  params:
    dashboard_component:
      dash_component:
        name: ListComposite
        module: __main__
        params:
          list_factory:
            dash_figure_factory:
              name: ListFactory
              module: __main__
              params:
                list_input:
                - this
                - is
                - a
                - dumb
                - example
          first_n1: 2
          first_n2: 3
    port: 8050
    mode: dash
    kwargs: {}



In [ ]:
db.to_yaml("dashboard.yaml")

In [ ]:
db2 = DashApp.from_yaml("dashboard.yaml")

In [ ]:
if run_app:
    db2.run()

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_Example.ipynb.
Converted index.ipynb.
